In [ ]:
# Args
in_folder = "/content/SRN/"
out_folder = "/content/SRN/"

hf_repo_name = "MVP-Group-Project/srn_cars_priors"

git_token = ""
hf_token = ""

In [ ]:
!cd /content/
!git clone https://"{git_token}"@github.com/Kacper-M-Michalik/3dgs-priors.git

In [ ]:
import os
import glob
from huggingface_hub import login, HfApi
from datasets import load_dataset

In [ ]:
# Create venv for each prior model
!python3 -m venv /content/models/depth --without-pip
!python3 -m venv /content/models/normal --without-pip

# Have to manually install pip to correctly build venvs on colab
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/models/depth/bin/python3 get-pip.py
!/content/models/normal/bin/python3 get-pip.py

# Verify venv's work
!/content/models/depth/bin/pip --version
!/content/models/normal/bin/pip --version

!ls -l /content/models/

In [ ]:
# Setup models

# Depth
!/content/models/depth/bin/pip install -r /content/3dgs-priors/geometry-priors/depth_requirements.txt

# Normals
!git clone https://github.com/baegwangbin/surface_normal_uncertainty.git /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/
!mkdir /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/checkpoints/
!gdown 1lOgY9sbMRW73qNdJze9bPkM2cmfA8Re- -O /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/checkpoints/scannet.pt
!/content/models/normal/bin/pip install -r /content/3dgs-priors/geometry-priors/normal_requirements.txt

In [ ]:
# Get SRN cars dataset
%cd /content
!mkdir SRN
%cd /content/SRN
!mkdir srn_cars
%cd /content/SRN/srn_cars
!gdown --id 19yDsEJjx9zNpOKz9o6AaK-E8ED6taJWU -O cars.zip
!unzip cars.zip

In [ ]:
# Launch models to process dataset
!/content/models/normal/bin/python /content/3dgs-priors/geometry-priors/generate_normal.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=30

In [ ]:
!/content/models/depth/bin/python /content/3dgs-priors/geometry-priors/generate_depth.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=30


In [ ]:
# Upload generated dataset to HuggingFace
login(token=hf_token)

# Dataset follow ShapeNet layout, but are stored in Parquet format isntead of folders (due to performance/rate limiting reasons)
parquet_files = glob.glob(os.path.join(out_folder, "*.parquet"))
print("Found {} parquet files to upload".format(len(parquet_files)))

for file_path in parquet_files:
    name = os.path.basename(file_path).split('.')[0]    
    dataset = load_dataset("parquet", data_files=file_path)    
    dataset.push_to_hub(
        hf_repo_name, 
        config_name=name
    )